#book recommender system using collaborative clustering


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv

In [2]:
books=pd.read_csv("BX-Books.csv",sep=';',encoding='latin-1',error_bad_lines=False)

C:\Users\Tanmay Somani\AppData\Local\Temp\ipykernel_17276\3921296444.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books=pd.read_csv("BX-Books.csv",sep=';',encoding='latin-1',error_bad_lines=False)
Skipping line 6452: expected 8 fields, saw 9
Skipping line 43667: expected 8 fields, saw 10
Skipping line 51751: expected 8 fields, saw 9

Skipping line 92038: expected 8 fields, saw 9
Skipping line 104319: expected 8 fields, saw 9
Skipping line 121768: expected 8 fields, saw 9

Skipping line 144058: expected 8 fields, saw 9
Skipping line 150789: expected 8 fields, saw 9
Skipping line 157128: expected 8 fields, saw 9
Skipping line 180189: expected 8 fields, saw 9
Skipping line 185738: expected 8 fields, saw 9

Skipping line 209388: expected 8 fields, saw 9
Skipping line 220626: expected 8 fields, saw 9
Skipping line 227933: expected 8 fields, saw 11
Skipping line 228957: expected 8 fields, sa

In [3]:
books=books[['ISBN','Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher']]

In [4]:
books.rename(columns={
    "Book-Title":"title",
    "Books-Author":"author",
    "Year-Of-Publication":"year",
    "Image-URL-L":"img_url"
},inplace=True)

In [5]:
users=pd.read_csv("BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

C:\Users\Tanmay Somani\AppData\Local\Temp\ipykernel_17276\5631173.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  users=pd.read_csv("BX-Users.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In [6]:
ratings=pd.read_csv("BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1') 

C:\Users\Tanmay Somani\AppData\Local\Temp\ipykernel_17276\3966382842.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  ratings=pd.read_csv("BX-Book-Ratings.csv",sep=';',error_bad_lines=False ,encoding='latin-1')


In collaborative system it is important for a user to give you ratings.

In [7]:
ratings.rename(columns={"User-ID":"user_id","Book-Rating":"rating"},inplace=True)

In [8]:
x=ratings["user_id"].value_counts()>200

In [9]:
y=x[x].index

In [10]:
ratings=ratings[ratings['user_id'].isin(y)]

In [11]:
ratings_with_books=ratings.merge(books,on="ISBN")

In [12]:
num_rating=ratings_with_books.groupby('title')['rating'].count().reset_index()

In [13]:
num_rating.rename(columns={"rating":"num_of_rating"},inplace=True)

In [14]:
final_rating=ratings_with_books.merge(num_rating,on ='title')

In [15]:
final_rating=final_rating[final_rating["num_of_rating"]>=50]

In [16]:
final_rating.sample(10)

,user_id,ISBN,rating,title,Book-Author,year,Publisher,num_of_rating
110023,246311,0452269571,0,Bastard Out of Carolina,Dorothy Allison,1993,Plume Books,62
7719,15408,0440241073,0,The Summons,John Grisham,2002,Dell Publishing Company,171
82228,171118,0553381342,0,The Bonfire of the Vanities,TOM WOLFE,2001,Bantam,92
39561,49109,0451156609,0,The Tommyknockers,Stephen King,1994,Signet Book,98
60279,53174,0316690619,2,Cradle and All,James Patterson,2000,Little Brown and Company,109
125681,11601,0425162443,0,The Rapture of Canaan,Sheri Reynolds,1997,Berkley Publishing Group,87
91992,254465,0451203895,0,4 Blondes,Candace Bushnell,2002,Signet Book,71
62105,141902,0679429220,0,Midnight in the Garden of Good and Evil: A Sav...,John Berendt,1994,Random House,89
133168,52584,0425132048,0,Eyes of Prey,John Sandford,2004,Berkley Publishing Group,50
4454,6543,0385504209,0,The Da Vinci Code,Dan Brown,2003,Doubleday,224


In [17]:
final_rating.drop_duplicates(['user_id','title'],inplace=True)

In [18]:
book_pivot=final_rating.pivot_table(columns='user_id',index='title',values='rating')

In [19]:
book_pivot.fillna(0,inplace=True)

In [20]:
from scipy.sparse import csr_matrix

In [21]:
book_sparse=csr_matrix(book_pivot)

In [22]:
from sklearn.neighbors import NearestNeighbors

In [23]:
model=NearestNeighbors(algorithm='brute')


In [24]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [25]:
distance,suggestion=model.kneighbors(book_pivot.iloc[237,:].values.reshape(1,-1),n_neighbors=6)


In [26]:
distance

array([[ 0.        , 68.78953409, 69.5413546 , 72.64296249, 76.83098333,
        77.28518616]])

In [27]:
suggestion

array([[237, 240, 238, 241, 184, 536]], dtype=int64)

In [28]:
for i in range(len(suggestion)):
  print(book_pivot.index[suggestion[i]])
  #to see the recommendation

Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [29]:
book_pivot.index[3]

'4 Blondes'

In [30]:
books_name=book_pivot.index

In [31]:
import pickle
pickle.dump(model,open('artifacts/model.pkl','wb'))
pickle.dump(books_name,open('artifacts/book_name.pkl','wb'))
pickle.dump(final_rating,open('artifacts/final_rating.pkl','wb'))
pickle.dump(book_pivot,open('artifacts/book_pivot.pkl','wb'))


In [32]:
def recommend_me_the_book(book_name):
  try:
    book_id=np.where(book_pivot.index ==book_name)[0][0]
    distance,suggestion=model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors=6)
  
    for i in range(len(suggestion)):
      books=book_pivot.index[suggestion[i]]
      for j in books:
        print(j)
  except IndexError:
    print("The book doesnt exists in the database")
  


In [33]:
book_name=''
recommend_me_the_book(book_name)

The book doesnt exists in the database
